### This file tunes the alpha parameter for our text mining svm model
#### sample with replacement 1000 times with a five-folder cross validation in order to find the optimal tuning parameter, alpha 
#### optimal alpha equals to 3.10270270e-05




In [1]:
import numpy as np
import time
import imp
import sklearn as sk
import random
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import json
import csv
import unicodecsv
import codecs
from collections import defaultdict
import sys
import lda
import csv
#imp.reload(sys)
#sys.stdout = codecs.getwriter('utf-8')(sys.stdout)
#sys.stderr = codecs.getwriter('utf-8')(sys.stderr)
import pickle
from collections import Counter
import itertools
import nltk
import re
import math
from nltk.probability import *
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
import pandas as pd
from sklearn.

In [2]:
# train and test on "final_pos_neg_sentences.csv" 
review_list = []
with open('/Users/Ellie/Documents/BERKELEY/STAT154/Proj/data_updata/yelp_academic_dataset_review_train.csv', encoding = 'utf8') as f:
    reader = csv.reader(f, delimiter=',')
    for line in reader:
        review_list.append(tuple(line))
        
review_list.pop(0)

('',
 'funny',
 'user_id',
 'review_id',
 'text',
 'business_id',
 'stars',
 'date',
 'useful',
 'type',
 'cool')

In [3]:
# perform cross validation
cv_id = []
for i in range(0, 5):
    cv_id.extend([i] * 502)
random.shuffle(cv_id)

In [ ]:
value = []
p = []
all_business_id = np.unique(business_id)
for val in np.linspace(10 ** (-3), 10 ** (-7), 1000): 
    mse = []
    for i in range(0,5):
        train_business = [j for j in range(1, len(all_business_id)) if cv_id[j] != i]
        test_buisness = [j for j in range(1, len(all_business_id)) if cv_id[j] == i]
        train_id = [all_business_id[i - 1] for i in train_business]
        test_id = [all_business_id[i - 1] for i in test_buisness]
        train_index = [i for i in range(1, len(review_list)-1) if review_list[i][5] in train_id]
        test_index = [i for i in range(1, len(review_list) -1) if review_list[i][5] in test_id]

        train_list = [review_list[j][4] for j in train_index]
        train_response = [review_list[j][6] for j in train_index]
        #     train_list_id = [element[5] for element in train_set]
        test_list = [review_list[j][4] for j in test_index]
        test_response = [review_list[j][6] for j in test_index]
        test_list_id = [review_list[j][5] for j in test_index]
        
        ## SVM model 
        text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2', alpha = val, n_iter=5, random_state=42))])
        text_clf = text_clf.fit(train_list, train_response)
        pred = text_clf.predict(test_list)
        pred = pred.astype(int)
        result = pd.DataFrame({'business_id': test_list_id, 'stars': pred})
        result_agg = result.groupby(by = ['business_id'])['stars'].mean()
        result_agg = result_agg.reset_index()

        result_act = pd.DataFrame({'business_id': test_list_id, 'stars': list(map(int, test_response))})
        result_act = result_act.groupby(by = ['business_id'])['stars'].mean()
        result_act = result_act.reset_index()    
        combine = pd.merge(result_agg, result_act, on = 'business_id')
        mse.append(np.mean((combine['stars_x'] - combine['stars_y'])**2))
    value.append(np.mean(mse))
    p.append(val)
    



In [ ]:
plt.plot(p, mse)
plt.xlabel('alpha')
plt.ylabel('MSE')
plt.title("CV MSE v.s. Alpha")
plt.show()

In [ ]:
# The whole data set
train_set = [element[4] for element in review_list]
train_response = [element[6] for element in review_list]
train_list_id = [element[5] for element in review_list]
test_set = [element[4] for element in test_list]
test_list_id = [element[5] for element in test_list]

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2', alpha = 3.10270270e-05, n_iter=5, random_state=42))])
text_clf = text_clf.fit(train_set, train_response)
pred = text_clf.predict(test_set)
pred = pred.astype(int)
result = pd.DataFrame({'business_id': test_list_id, 'stars': pred})
result_agg = result.groupby(by = ['business_id'])['stars'].mean()
result_agg = result_agg.reset_index()
result_agg.to_csv("outtext.csv", index=False)
